In [1]:
%run stdPackages.ipynb
slides = False # print to slides format if True
out_folder = os.path.join(d['curr'], 'Misc', 'Figs')
d['data'] = os.path.join(d['curr'],'Misc','Data')
read = {'variables': ['Fundamentals', 'Load', 'Generators_Other'], 
        'variable2D': ['Generators_FuelMix','HourlyVariation'],
        'scalars': ['MWP'],
        'maps': ['Generators_Categories']}
db = dbFromWB(os.path.join(d['data'],'mBasicInt1.xlsx'), read)
db.updateAlias(alias=[('h','h_alias')])
readSets(db)

Set up model:

In [2]:
m = mBasicInt.mBasicInt(db)
m.solve()

Solution status 0: Optimization terminated successfully.


Load duration curve (LDC) and residual demand curve (RDC):

In [3]:
LDC = m.hourlyLoad.sort_values(ascending=False)
LDC.index = [i/(len(LDC)) for i in range(1, len(LDC)+1)]
LDC.at[0] = LDC.iloc[0]
RDC = (m.hourlyLoad-m.hourlyGeneratingCapacity.xs('id4')).sort_values(ascending = False)
RDC.index = [i/(len(RDC)) for i in range(1, len(RDC)+1)]
RDC.at[0] = RDC.iloc[0]

## Simple plots:

### LDC and RDC:

In [4]:
%%capture
mult_graphs()
nplots = 4
nrows = math.ceil(nplots/2)
fig, axes = plt.subplots(nrows, min(nplots, 2), figsize = (14, (4*nrows)));
plt.subplots_adjust(hspace=0.35)
# Plot 1: Hourly load
ax = plt.subplot(nrows, min(nplots,2), 1)
m.hourlyLoad.plot.bar(ax = ax, legend = False);
ax.set_xlabel(r'$h$', labelpad=10);
ax.set_ylabel(r"Load, GJ", labelpad=10);
ax.set_title('Hourly load');

# Plot 2: Load duration curve
ax = plt.subplot(nrows, min(nplots,2), 2)
seaborn.lineplot(data=LDC, ax = ax, linewidth = 3, legend = False);
ax.set_xlim([0,1]);
ax.set_xlabel(r'Capacity factor', labelpad=10);
ax.set_ylabel(r"Load, GJ",labelpad=10);
ax.set_title('Load duration curve');

# Plot 3: Hourly Variation in Generation:
ax = plt.subplot(nrows, min(nplots,2), 3)
m.hourlyGeneratingCapacity.xs('id4').plot.bar(ax = ax, legend = False);
ax.set_xlabel(r'$h$', labelpad=10);
ax.set_ylabel(r"Generating capacity, GJ", labelpad=10);
ax.set_title('Generating capacity, wind');

# Plot 4: Residual demand curve
ax = plt.subplot(nrows, min(nplots,2), 4)
seaborn.lineplot(data=RDC, ax = ax, linewidth = 3, legend = False);
ax.set_xlim([0,1]);
ax.set_xlabel(r'Capacity factor', labelpad=10);
ax.set_ylabel(r"Demand net of wind, GJ",labelpad=10);
ax.set_title('Residual demand curve');

fig.tight_layout()
if slides:
    fig.savefig(f"{out_folder}\\mBasicInt_hvt_slides.pdf",facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(f"{out_folder}\\mBasicInt_hvt.pdf",edgecolor='k')

### Plot merit order curve for each hour:

In [11]:
def demand_h(m,h):
    return standardPlots.demandLinear_df(m.db['MWP_LoadShedding'], m.hourlyLoad.xs(h))
def supply_h(m,h,maxY=30):
    return standardPlots.meritOrderCurve_df(m.db['mc'], m.hourlyGeneratingCapacity.xs(h,level='h'),maxY=maxY)

In [18]:
%%capture
mult_graphs()
nplots = len(m.db['h'])
nrows = math.ceil(nplots/2)
fig,axes = plt.subplots(nrows,min(nplots,2),figsize=(14,(4*nrows)));
plt.subplots_adjust(hspace=0.35)
offset_N, offset_Arrow, offset_q = 1, 1, 3
for j in range(nplots):
    ax = plt.subplot(nrows,min(nplots,2),j+1)
    h = j+1
    df = supply_h(m,h)
    dh = demand_h(m,h)
    
    df.plot(linewidth=3,ax=ax,legend=False);
    dh.plot(linewidth=3,ax=ax,legend=False);
    ax.set_xlabel(r'$GJ$', labelpad = 5);
    ax.set_ylabel(r'$€/$GJ', labelpad = 5);
    ax.set_xlim([0, max([dh.index.max(), df.index.max()])+5]);
    ax.set_ylim([0, math.ceil(max([df.max()[0], dh.max()[0]]))]);
    ax.set_title(f"""$h={h}$""")
    
    plt.text(df.index[2]/2-8, df.iloc[2]+offset_N, f'Wind');
    plt.text(df.index[2]/2-5, df.iloc[2]-offset_q, f'$q_W$');
    plt.annotate("",xy=(1, df.iloc[2]-offset_Arrow), xytext=(df.index[2]-1, df.iloc[2]-offset_Arrow), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));
    plt.annotate("",xy=(df.index[2]-1, df.iloc[2]-offset_Arrow), xytext=(1, df.iloc[2]-offset_Arrow), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));
    
    plt.text((df.index[4]+df.index[2])/2-8, df.iloc[4]+offset_N, f'Coal')
    plt.text((df.index[4]+df.index[2])/2-5, df.iloc[4]-offset_q, f'$q_C$');
    plt.annotate("",xy=(df.index[2]+1, df.iloc[4]-offset_Arrow), xytext=(df.index[4]-1,df.iloc[4]-offset_Arrow), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));
    plt.annotate("",xy=(df.index[4]-1, df.iloc[4]-offset_Arrow), xytext=(df.index[2]+1, df.iloc[4]-offset_Arrow), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));
    
    plt.text((df.index[6]+df.index[4])/2-6, df.iloc[6]+offset_N, f'Gas')
    plt.text((df.index[6]+df.index[4])/2-5, df.iloc[6]-offset_q, f'$q_C$');
    plt.annotate("",xy=(df.index[4]+1, df.iloc[6]-offset_Arrow), xytext=(df.index[6]-1,df.iloc[6]-offset_Arrow), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));
    plt.annotate("",xy=(df.index[6]-1, df.iloc[6]-offset_Arrow), xytext=(df.index[4]+1, df.iloc[6]-offset_Arrow), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));
    
    plt.text((df.index[8]+df.index[6])/2-6, df.iloc[8]+offset_N, f'Bio')
    plt.text((df.index[8]+df.index[6])/2-5, df.iloc[8]-offset_q, f'$q_B$');
    plt.annotate("",xy=(df.index[6]+1, df.iloc[8]-offset_Arrow), xytext=(df.index[8]-1,df.iloc[8]-offset_Arrow), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));
    plt.annotate("",xy=(df.index[8]-1, df.iloc[8]-offset_Arrow), xytext=(df.index[6]+1, df.iloc[8]-offset_Arrow), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));

fig.legend(['Generation', 'Load'],loc=9,ncol=2,frameon=True)
fig.tight_layout();
fig.subplots_adjust(top=0.9);
if slides:
    fig.savefig(f"{out_folder}\\mBasicInt_MeritOrder_slides.pdf",facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(f"{out_folder}\\mBasicInt_MeritOrder.pdf",edgecolor='k')